In [1]:
!pip install pygame mediapipe

In [2]:
import cv2
import mediapipe as mp
import numpy as np

# MediaPipe 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# 카메라 설정
cap = cv2.VideoCapture(0)

# 공 설정
ball_pos = np.array([320, 240])
ball_vel = np.array([0, 0])
ball_radius = 15
gravity = 0.5

# 점수 초기화
score = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    # 손가락 위치 찾기
    ball_touched = False
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            for id, lm in enumerate(hand_landmarks.landmark):
                if id == 8:  # 검지 손가락 끝
                    finger_tip = np.array([lm.x, lm.y]) * [frame.shape[1], frame.shape[0]]
                    if np.linalg.norm(finger_tip - ball_pos) < ball_radius + 20:
                        ball_vel = (ball_pos - finger_tip) * 0.1
                        ball_vel[1] -= 5  # 손가락에 닿으면 위로 튕겨나감
                        ball_touched = True

    # 공의 물리적 움직임 시뮬레이션
    ball_vel[1] += gravity  # 중력 적용
    ball_pos = ball_pos + ball_vel
    ball_pos = ball_pos.astype(int)  # ball_pos를 정수형으로 변환

    # 화면 범위 내에서 공 유지
    if ball_pos[0] < ball_radius or ball_pos[0] > frame.shape[1] - ball_radius:
        ball_vel[0] *= -1
    if ball_pos[1] < ball_radius:
        ball_vel[1] *= -1

    # 바닥에 닿으면 게임 종료
    if ball_pos[1] > frame.shape[0] - ball_radius:
        break

    # 점수 업데이트
    if ball_touched:
        score += 1

    # 공 그리기
    cv2.circle(frame, tuple(ball_pos.astype(int)), ball_radius, (0, 255, 0), -1)

    # 점수 표시
    cv2.putText(frame, f'Score: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # 화면에 결과 표시
    cv2.imshow('손가락으로 공 튕기기', frame)

    # 'q'를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료 처리
print(f"Final Score: {score}")
cap.release()
cv2.destroyAllWindows()


Final Score: 0


In [1]:
import cv2
import mediapipe as mp
import numpy as np

# MediaPipe 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7)

# 카메라 설정
cap = cv2.VideoCapture(0)

# 점수 초기화
score = 0

# 공 위치 및 속도 설정
ball_pos = np.array([320, 240])
ball_vel = np.array([0, 0])
ball_radius = 15
gravity = 0.5

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    # 손가락 위치 찾기
    ball_touched = False
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 검지 손가락 끝의 랜드마크만 찾기
            index_finger_tip = hand_landmarks.landmark[8]
            finger_tip = np.array([index_finger_tip.x, index_finger_tip.y]) * [frame.shape[1], frame.shape[0]]
            cv2.circle(frame, (int(finger_tip[0]), int(finger_tip[1])), 10, (255, 0, 0), -1)

            # 검지 손가락 끝과 공이 닿았는지 확인
            if np.linalg.norm(finger_tip - ball_pos) < ball_radius + 20:
                ball_vel = (ball_pos - finger_tip) * 0.1
                ball_vel[1] -= 8  # 손가락에 닿으면 위로 튕겨나감
                ball_touched = True

    # 공의 물리적 움직임 시뮬레이션
    ball_vel[1] += gravity  # 중력 적용
    ball_pos = ball_pos + ball_vel
    ball_pos = ball_pos.astype(int)  # ball_pos의 값을 정수형으로 명시적 캐스팅
    ball_pos = np.clip(ball_pos, ball_radius, np.array([frame.shape[1], frame.shape[0]]) - ball_radius)


    # 화면 범위 내에서 공 유지
    if ball_pos[0] <= ball_radius or ball_pos[0] >= frame.shape[1] - ball_radius:
        ball_vel[0] *= -1  # 벽에 닿으면 X축 방향의 속도를 반전
    if ball_pos[1] <= ball_radius: 
        ball_vel[1] *= -1  # 천장에 닿으면 Y축 방향의 속도를 반전

    # 점수 업데이트
    if ball_touched:
        score += 1

    # 바닥에 닿으면 게임 종료
    if ball_pos[1] >= frame.shape[0] - ball_radius:
        print(f"Game Over! Final Score: {score}")  # 게임 오버 메시지 출력
        break

    # 공 그리기
    cv2.circle(frame, tuple(ball_pos.astype(int)), ball_radius, (0, 255, 0), -1)

    # 점수 표시
    cv2.putText(frame, f'Score: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # 화면에 결과 표시
    cv2.imshow('손가락으로 공 튕기기', frame)

    # 'q'를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료 처리
cap.release()
cv2.destroyAllWindows()


In [38]:
import cv2
import mediapipe as mp
import numpy as np
import time
import random

# MediaPipe 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.1)

# 카메라 설정
cap = cv2.VideoCapture(0)

# 점수 및 게임 시작 시간 초기화
score = 0
start_time = time.time()

# 첫 번째 공 위치 및 속도 설정
ball_pos = np.array([220, 0])
ball_vel = np.array([0, 5])  # 초기 수직 속도 설정
ball_radius = 15
gravity = 0.5

# 두 번째 공 관련 변수
second_ball_active = False
second_ball_pos = np.array([0, 0])
second_ball_vel = np.array([0, 0])

# 장애물 설정
obstacle_size = np.array([150, 10])  # 장애물의 크기 (가로, 세로)
obstacle_pos = np.array([random.randint(0, 640 - obstacle_size[0]), 
                         random.randint(0, 480 - obstacle_size[1])])

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 현재 시간
    current_time = time.time()

    # 10초 후 두 번째 공 활성화 및 초기 속도 설정
    if not second_ball_active and current_time - start_time >= 10:
        second_ball_active = True
        second_ball_pos = np.array([320, 120])  # 적당한 시작 위치 설정
        second_ball_vel = np.array([0, 5])  # 초기 수직 속도 설정

    frame = cv2.flip(frame, 1)  # 좌우 반전
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    # 손가락 위치 찾기
    ball_touched = False
    second_ball_touched = False
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 검지 손가락 끝의 랜드마크만 찾기
            index_finger_tip = hand_landmarks.landmark[8]
            finger_tip = np.array([index_finger_tip.x, index_finger_tip.y]) * [frame.shape[1], frame.shape[0]]
            cv2.circle(frame, (int(finger_tip[0]), int(finger_tip[1])), 10, (255, 0, 0), -1)

            # 검지 손가락 끝과 첫 번째 공이 닿았는지 확인
            if np.linalg.norm(finger_tip - ball_pos) < ball_radius + 20:
                ball_vel = (ball_pos - finger_tip) * 0.1
                ball_vel[1] -= 8
                if not ball_touched:
                    score += 1
                    print(f"Score: {score}")  # 점수 증가 확인을 위한 로그
                ball_touched = True

            # 두 번째 공이 활성화되었을 경우, 검지 손가락과의 충돌 확인
            if second_ball_active and np.linalg.norm(finger_tip - second_ball_pos) < ball_radius + 20:
                second_ball_vel = (second_ball_pos - finger_tip) * 0.1
                second_ball_vel[1] -= 8
                if not second_ball_touched:
                    score += 1
                    print(f"Score: {score}")  # 점수 증가 확인을 위한 로그
                second_ball_touched = True

    # 첫 번째 공의 물리적 움직임 시뮬레이션
    ball_vel[1] += gravity
    ball_pos = (ball_pos + ball_vel).astype(int)
    ball_pos = np.clip(ball_pos, ball_radius, np.array([frame.shape[1], frame.shape[0]]) - ball_radius)

    if ball_pos[0] <= ball_radius or ball_pos[0] >= frame.shape[1] - ball_radius:
        ball_vel[0] *= -1
    if ball_pos[1] <= ball_radius:
        ball_vel[1] *= -1

    # 두 번째 공의 물리적 움직임 시뮬레이션
    if second_ball_active:
        second_ball_vel[1] += gravity
        second_ball_pos = (second_ball_pos + second_ball_vel).astype(int)
        second_ball_pos = np.clip(second_ball_pos, ball_radius, np.array([frame.shape[1], frame.shape[0]]) - ball_radius)

        if second_ball_pos[0] <= ball_radius or second_ball_pos[0] >= frame.shape[1] - ball_radius:
            second_ball_vel[0] *= -1
        if second_ball_pos[1] <= ball_radius:
            second_ball_vel[1] *= -1

    # 장애물과의 충돌 검사 및 점수 업데이트
    obstacle_rect = [obstacle_pos[0], obstacle_pos[1], obstacle_pos[0] + obstacle_size[0], obstacle_pos[1] + obstacle_size[1]]
    if (ball_pos[0] + ball_radius > obstacle_rect[0] and ball_pos[0] - ball_radius < obstacle_rect[2] and
        ball_pos[1] + ball_radius > obstacle_rect[1] and ball_pos[1] - ball_radius < obstacle_rect[3]):
        score -= 1

    if second_ball_active and (
        second_ball_pos[0] + ball_radius > obstacle_rect[0] and second_ball_pos[0] - ball_radius < obstacle_rect[2] and
        second_ball_pos[1] + ball_radius > obstacle_rect[1] and second_ball_pos[1] - ball_radius < obstacle_rect[3]):
        score -= 1

    # 첫 번째 공 그리기
    cv2.circle(frame, tuple(ball_pos), ball_radius, (255, 255, 255), -1)

    # 두 번째 공 그리기
    if second_ball_active:
        cv2.circle(frame, tuple(second_ball_pos), ball_radius, (255, 255, 255), -1)

    # 장애물 그리기 (사각형)
    cv2.rectangle(frame, (obstacle_pos[0], obstacle_pos[1]), (obstacle_pos[0] + obstacle_size[0], obstacle_pos[1] + obstacle_size[1]), (0, 0, 255), -1)

    # 점수 표시
    cv2.putText(frame, f'Score: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # 화면에 결과 표시
    cv2.imshow('손가락으로 공 튕기기', frame)

    # 'q'를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # 바닥에 닿으면 게임 종료
    if ball_pos[1] >= frame.shape[0] - ball_radius or (second_ball_active and second_ball_pos[1] >= frame.shape[0] - ball_radius):
        print(f"Game Over! Final Score: {score}")
        break

# 종료 처리
cap.release()
cv2.destroyAllWindows()


Score: -9
Score: -8
Score: -7
Game Over! Final Score: -7


----

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import time
import random

# MediaPipe 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.1)

# 카메라 설정
cap = cv2.VideoCapture(0)

# 점수 및 게임 시작 시간 초기화
score = 0
start_time = time.time()

# 공의 이미지 로드 및 크기 조정
ball_image_path = 'soccerball.png'  # 이미지 경로를 정확히 설정하세요.
ball_image = cv2.imread(ball_image_path, -1)  # -1 플래그로 알파 채널 포함 읽기
ball_image = cv2.resize(ball_image, (30, 30))  # 이미지 크기를 공의 크기로 조정
ball_radius = 15

# 첫 번째 공 위치 및 속도 설정
ball_pos = np.array([220, 0])
ball_vel = np.array([0, 5])  # 초기 수직 속도 설정
gravity = 0.5

# 두 번째 공 관련 변수
second_ball_active = False
second_ball_pos = np.array([320, 0])
second_ball_vel = np.array([0, 5])

# 장애물 설정
obstacle_size = np.array([150, 10])  # 장애물의 크기 (가로, 세로)
obstacle_pos = np.array([random.randint(0, 640 - obstacle_size[0]), 
                         random.randint(0, 480 - obstacle_size[1])])

# 이미지의 알파 채널로 마스크 생성
ball_mask = ball_image[:, :, 3]
ball_mask_inv = cv2.bitwise_not(ball_mask)
ball_image = ball_image[:, :, :3]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 현재 시간
    current_time = time.time()

    # 10초 후 두 번째 공 활성화 및 초기 속도 설정
    if not second_ball_active and current_time - start_time >= 10:
        second_ball_active = True
        second_ball_pos = np.array([320, 0])  # 적당한 시작 위치 설정
        second_ball_vel = np.array([0, 5])  # 초기 수직 속도 설정

    frame = cv2.flip(frame, 1)  # 좌우 반전
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    # 손가락 위치 찾기
    ball_touched = False
    second_ball_touched = False
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 검지 손가락 끝의 랜드마크만 찾기
            index_finger_tip = hand_landmarks.landmark[8]
            finger_tip = np.array([index_finger_tip.x, index_finger_tip.y]) * [frame.shape[1], frame.shape[0]]
            cv2.circle(frame, (int(finger_tip[0]), int(finger_tip[1])), 10, (255, 0, 0), -1)

            # 검지 손가락 끝과 첫 번째 공이 닿았는지 확인
            if np.linalg.norm(finger_tip - ball_pos) < ball_radius + 20:
                ball_vel = (ball_pos - finger_tip) * 0.1
                ball_vel[1] -= 8
                if not ball_touched:
                    score += 1
                    print(f"Score: {score}")  # 점수 증가 확인을 위한 로그
                ball_touched = True

            # 두 번째 공이 활성화되었을 경우, 검지 손가락과의 충돌 확인
            if second_ball_active and np.linalg.norm(finger_tip - second_ball_pos) < ball_radius + 20:
                second_ball_vel = (second_ball_pos - finger_tip) * 0.1
                second_ball_vel[1] -= 8
                if not second_ball_touched:
                    score += 1
                    print(f"Score: {score}")  # 점수 증가 확인을 위한 로그
                second_ball_touched = True

    # 첫 번째 공의 물리적 움직임 시뮬레이션
    ball_vel[1] += gravity
    ball_pos = (ball_pos + ball_vel).astype(int)
    ball_pos = np.clip(ball_pos, ball_radius, np.array([frame.shape[1], frame.shape[0]]) - ball_radius)

    # 두 번째 공의 물리적 움직임 시뮬레이션
    if second_ball_active:
        second_ball_vel[1] += gravity
        second_ball_pos = (second_ball_pos + second_ball_vel).astype(int)
        second_ball_pos = np.clip(second_ball_pos, ball_radius, np.array([frame.shape[1], frame.shape[0]]) - ball_radius)

        if second_ball_pos[0] <= ball_radius or second_ball_pos[0] >= frame.shape[1] - ball_radius:
            second_ball_vel[0] *= -1
        if second_ball_pos[1] <= ball_radius:
            second_ball_vel[1] *= -1

    # 첫 번째 공 그리기
    if 0 <= ball_pos[1] - ball_radius and ball_pos[1] + ball_radius < frame.shape[0] and 0 <= ball_pos[0] - ball_radius and ball_pos[0] + ball_radius < frame.shape[1]:
        ball_center = (ball_pos[0] - ball_radius, ball_pos[1] - ball_radius)
        ball_rect = frame[ball_center[1]:ball_center[1]+2*ball_radius, ball_center[0]:ball_center[0]+2*ball_radius]
        ball_alpha_s = ball_mask / 255.0
        ball_alpha_l = 1.0 - ball_alpha_s

        for c in range(0, 3):
            ball_rect[:, :, c] = (ball_alpha_s * ball_image[:, :, c] +
                                ball_alpha_l * ball_rect[:, :, c])

    # 두 번째 공 그리기
    if second_ball_active and 0 <= second_ball_pos[1] - ball_radius and second_ball_pos[1] + ball_radius < frame.shape[0] and 0 <= second_ball_pos[0] - ball_radius and second_ball_pos[0] + ball_radius < frame.shape[1]:
        ball_center = (second_ball_pos[0] - ball_radius, second_ball_pos[1] - ball_radius)
        ball_rect = frame[ball_center[1]:ball_center[1]+2*ball_radius, ball_center[0]:ball_center[0]+2*ball_radius]
        ball_alpha_s = ball_mask / 255.0
        ball_alpha_l = 1.0 - ball_alpha_s

        for c in range(0, 3):
            ball_rect[:, :, c] = (ball_alpha_s * ball_image[:, :, c] +
                                ball_alpha_l * ball_rect[:, :, c])

    # 장애물 그리기 (사각형)
    cv2.rectangle(frame, tuple(obstacle_pos), tuple(obstacle_pos + obstacle_size), (0, 0, 255), -1)

    # 점수 표시
    cv2.putText(frame, f'Score: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # 화면에 결과 표시
    cv2.imshow('손가락으로 공 튕기기', frame)

    # 'q'를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # 바닥에 닿으면 게임 종료
    if ball_pos[1] >= frame.shape[0] - ball_radius or (second_ball_active and second_ball_pos[1] >= frame.shape[0] - ball_radius):
        print(f"Game Over! Final Score: {score}")
        break

# 종료 처리
cap.release()
cv2.destroyAllWindows()


TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [8]:
!pip install protobuf==3.20.0

In [7]:
import cv2
import mediapipe as mp
import numpy as np
import time
import random

# MediaPipe 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5)

# 플레이어 닉네임 입력
player_name = input("Enter a nickname for the game: ")

# 카메라 설정
cap = cv2.VideoCapture(0)

# 점수 및 게임 시작 시간 초기화
score = 0
start_time = time.time()

# 공의 이미지 로드 및 크기 조정
ball_image_path = 'soccerball.png'  
ball_image = cv2.imread(ball_image_path, -1)  # -1 플래그로 알파 채널 포함 읽기
if ball_image is None:
    print(f"Failed to load image at {ball_image_path}")
    exit()
ball_image = cv2.resize(ball_image, (30, 30))  # 이미지 크기를 공의 크기로 조정
ball_radius = 15

# 첫 번째 공 위치 및 속도 설정
ball_pos = np.array([320, 0])
ball_vel = np.array([0, 5])
gravity = 0.5

# 두 번째 공 관련 변수
second_ball_active = False
second_ball_pos = np.array([0, 0])
second_ball_vel = np.array([0, 0])

# 첫 번째 장애물 설정
obstacle1_pos = np.array([random.randint(100, 540), random.randint(100, 380)])
obstacle1_radius = 30

# 두 번째 장애물 설정
obstacle2_pos = np.array([random.randint(100, 540), random.randint(100, 380)])
obstacle2_radius = 30

# 이미지의 알파 채널로 마스크 생성
ball_mask = ball_image[:, :, 3]
ball_mask_inv = cv2.bitwise_not(ball_mask)
ball_image = ball_image[:, :, :3]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 현재 시간
    current_time = time.time()

    # 10초 후 두 번째 공 활성화
    if not second_ball_active and current_time - start_time >= 10:
        second_ball_active = True
        second_ball_pos = np.array([random.randint(ball_radius, frame.shape[1] - ball_radius), 0])  # 화면 내에서 시작 위치 설정
        second_ball_vel = np.array([0, 5])  # 초기 속도 설정

    frame = cv2.flip(frame, 1)  # 좌우 반전
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    # 손가락 위치 찾기
    ball_touched = False
    second_ball_touched = False
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 검지 손가락 끝의 랜드마크만 찾기
            index_finger_tip = hand_landmarks.landmark[8]
            finger_tip = np.array([index_finger_tip.x, index_finger_tip.y]) * [frame.shape[1], frame.shape[0]]
            cv2.circle(frame, (int(finger_tip[0]), int(finger_tip[1])), 10, (255, 0, 0), -1)

            # 검지 손가락 끝과 첫 번째 공이 닿았는지 확인 (충돌 원리 : 거리)
            if np.linalg.norm(finger_tip - ball_pos) < ball_radius + 20:
                ball_vel = (ball_pos - finger_tip) * 0.1
                ball_vel[1] -= 8
                if not ball_touched:
                    score += 1
                ball_touched = True

                # np.linalg.norm(finger_tip - ball_pos)는 NumPy의 linalg.norm 함수를 사용하여 손가락 위치 (finger_tip)와 공의 위치 (ball_pos) 간의 거리를 계산합니다.

                # ball_radius + 20은 공의 반지름에 추가적인 여유를 더한 값입니다. 이 값보다 작다면 손가락과 공이 충돌했다고 판단합니다.

                # 충돌이 감지되면 ball_vel에 새로운 속도를 할당합니다. ball_pos - finger_tip는 손가락에서 공으로 향하는 벡터를 나타냅니다. 이 벡터에 0.1을 곱한 값이 새로운 공의 속도가 됩니다.

                # ball_vel[1] -= 8은 수직 방향(y 축)으로 속도를 추가로 감소시키는 부분입니다. 이것은 아마도 중력을 모방한 것으로 보입니다.

                # if not ball_touched:는 이전에 충돌이 감지되지 않았을 때에만 점수를 증가시킵니다. 이렇게 하면 한 번의 충돌로 여러 번의 점수 증가를 방지할 수 있습니다.

                # ball_touched = True는 현재 충돌이 감지되었음을 표시하는 변수입니다.

            # 두 번째 공이 활성화되었을 경우, 검지 손가락과의 충돌 확인
            if second_ball_active and np.linalg.norm(finger_tip - second_ball_pos) < ball_radius + 20:
                second_ball_vel = (second_ball_pos - finger_tip) * 0.1
                second_ball_vel[1] -= 8
                if not second_ball_touched:
                    score += 1
                second_ball_touched = True

    # 첫 번째 공의 물리적 움직임 시뮬레이션
    ball_vel[1] += gravity
    ball_pos = (ball_pos + ball_vel).astype(int)  # 여기에서 타입 변환
    
    # 벽 충돌 검사
    if ball_pos[0] <= ball_radius or ball_pos[0] >= frame.shape[1] - ball_radius:
        ball_vel[0] = -ball_vel[0]
    if ball_pos[1] <= ball_radius:
        ball_vel[1] = -ball_vel[1]
    ball_pos = np.clip(ball_pos, ball_radius, np.array([frame.shape[1], frame.shape[0]]) - ball_radius)

    # 두 번째 공의 물리적 움직임 시뮬레이션
    if second_ball_active:
        second_ball_vel[1] += gravity
        second_ball_pos = (second_ball_pos + second_ball_vel).astype(int)  # 여기에서 타입 변환
        second_ball_pos = np.clip(second_ball_pos, ball_radius, np.array([frame.shape[1], frame.shape[0]]) - ball_radius)
        # 벽 충돌 검사
        if second_ball_pos[0] <= ball_radius or second_ball_pos[0] >= frame.shape[1] - ball_radius:
            second_ball_vel[0] = -second_ball_vel[0]
        if second_ball_pos[1] <= ball_radius:
            second_ball_vel[1] = -second_ball_vel[1]
        second_ball_pos = np.clip(second_ball_pos, ball_radius, np.array([frame.shape[1], frame.shape[0]]) - ball_radius)


    # 장애물 그리기 (동그라미)
    cv2.circle(frame, tuple(obstacle1_pos), obstacle1_radius, (0, 0, 255), -1)
    cv2.circle(frame, tuple(obstacle2_pos), obstacle2_radius, (0, 0, 255), -1)

    # 장애물과의 충돌 검사 및 점수 업데이트 (첫 번째 장애물)
    if np.linalg.norm(ball_pos - obstacle1_pos) < ball_radius + obstacle1_radius:
        score -= 1
        ball_vel = -ball_vel  # 장애물에 닿으면 반대 방향으로 튕겨나감

    if second_ball_active and np.linalg.norm(second_ball_pos - obstacle1_pos) < ball_radius + obstacle1_radius:
        score -= 1
        second_ball_vel = -second_ball_vel  # 장애물에 닿으면 반대 방향으로 튕겨나감

    # 장애물과의 충돌 검사 및 점수 업데이트 (두 번째 장애물)
    if np.linalg.norm(ball_pos - obstacle2_pos) < ball_radius + obstacle2_radius:
        score -= 1
        ball_vel = -ball_vel

    if second_ball_active and np.linalg.norm(second_ball_pos - obstacle2_pos) < ball_radius + obstacle2_radius:
        score -= 1
        second_ball_vel = -second_ball_vel

    # 첫 번째 공 그리기
    if 0 <= ball_pos[1] - ball_radius and ball_pos[1] + ball_radius < frame.shape[0] and 0 <= ball_pos[0] - ball_radius and ball_pos[0] + ball_radius < frame.shape[1]:
        ball_center = (ball_pos[0] - ball_radius, ball_pos[1] - ball_radius)
        ball_rect = frame[ball_center[1]:ball_center[1]+2*ball_radius, ball_center[0]:ball_center[0]+2*ball_radius]
        ball_alpha_s = ball_mask / 255.0
        ball_alpha_l = 1.0 - ball_alpha_s

        for c in range(0, 3):
            ball_rect[:, :, c] = (ball_alpha_s * ball_image[:, :, c] +
                                ball_alpha_l * ball_rect[:, :, c])

    # 두 번째 공 그리기
        if 0 <= second_ball_pos[1] - ball_radius and second_ball_pos[1] + ball_radius < frame.shape[0] and 0 <= second_ball_pos[0] - ball_radius and second_ball_pos[0] + ball_radius < frame.shape[1]:
            ball_center = (second_ball_pos[0] - ball_radius, second_ball_pos[1] - ball_radius)
            ball_rect = frame[ball_center[1]:ball_center[1]+2*ball_radius, ball_center[0]:ball_center[0]+2*ball_radius]
            ball_alpha_s = ball_mask / 255.0
            ball_alpha_l = 1.0 - ball_alpha_s

            for c in range(0, 3):
                ball_rect[:, :, c] = (ball_alpha_s * ball_image[:, :, c] +
                                    ball_alpha_l * ball_rect[:, :, c])


    # 점수 표시
    cv2.putText(frame, f'{player_name}\'s score: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # 화면에 결과 표시
    cv2.imshow('Finger Ball Game', frame)

    # 'q'를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

 # 바닥에 닿으면 게임 종료
    if ball_pos[1] >= frame.shape[0] - ball_radius or (second_ball_active and second_ball_pos[1] >= frame.shape[0] - ball_radius):
        print(f"Game Over! {player_name}'s final score: {score}")
        cv2.putText(frame, 'Game Over', (frame.shape[1] // 2 - 100, frame.shape[0] // 2), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
        cv2.imshow('Finger Ball Game', frame)
        cv2.waitKey(2000)  # Wait for 2 seconds
        break

# 종료 처리
cap.release()
cv2.destroyAllWindows()


TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

-------------